# Assigning ambiguous counts


In [ ]:
import matplotlib
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import scvelo as scv
import scipy
import json
import os

with open('../../configs/config.json') as f:
    input_paths = json.load(f)
top_dir = input_paths['top_dir']
frydir = os.path.sep.join([top_dir, "results", "alevin_fry", "mouse_pancreas", "fry_knee_quant_usa_cr-like"])
e2n_path = os.path.sep.join([top_dir, "refs", "refdata-cellranger-mm10-2.1.0", "geneid_to_name.txt"])
os.makedirs("anndata", exist_ok= True)

In [ ]:
verbose = True
meta_info = json.load(open(os.path.sep.join([frydir, "meta_info.json"])))
ng = meta_info['num_genes']
usa_mode = meta_info['usa_mode']

if usa_mode:
    if verbose:
        print("processing input in USA mode, will return A+S as the spliced count, and U as the unspliced count")
else:
    print("please follow previous steps to generate the ount matrix in the USA mode")
    assert(False)

af_raw = sc.read_mtx(os.path.sep.join([frydir, "alevin", "quants_mat.mtx"]))
ng = int(ng/3)
e2n = dict([ l.rstrip().split() for l in open(e2n_path).readlines()])
var_names = [ l.rstrip() for l in open(os.path.sep.join([frydir, "alevin", "quants_mat_cols.txt"])).readlines()][:ng]
var_names = [e2n[e] for e in var_names]

obs_names = [ l.rstrip() for l in open(os.path.sep.join([frydir, "alevin", "quants_mat_rows.txt"])).readlines() ]

example_adata = scv.datasets.pancreas()


spliced = af_raw[:,range(0,ng)]
spliced.obs_names = obs_names
spliced.var_names = var_names
spliced.var_names_make_unique()
spliced = spliced[example_adata.obs_names, example_adata.var_names]

unspliced = af_raw[:,range(ng, 2*ng)]
unspliced.obs_names = obs_names
unspliced.var_names = var_names
unspliced.var_names_make_unique()
unspliced = unspliced[example_adata.obs_names, example_adata.var_names]

ambiguous = af_raw[:,range(2*ng,3*ng)]
ambiguous.obs_names = obs_names
ambiguous.var_names = var_names
ambiguous.var_names_make_unique()
ambiguous = ambiguous[example_adata.obs_names, example_adata.var_names]


spliced = pd.DataFrame.sparse.from_spmatrix(spliced.X, columns=spliced.var_names, index=spliced.obs_names).sparse.to_dense()
unspliced = pd.DataFrame.sparse.from_spmatrix(unspliced.X,columns=unspliced.var_names, index=unspliced.obs_names).sparse.to_dense()
ambiguous = pd.DataFrame.sparse.from_spmatrix(ambiguous.X,columns=ambiguous.var_names, index=ambiguous.obs_names).sparse.to_dense()

del(af_raw)

In [ ]:
spliced.sum().sum() / (spliced.sum().sum()+unspliced.sum().sum()+ambiguous.sum().sum())

In [ ]:
unspliced.sum().sum() / (spliced.sum().sum()+unspliced.sum().sum()+ambiguous.sum().sum())

In [ ]:
ambiguous.sum().sum() / (spliced.sum().sum()+unspliced.sum().sum()+ambiguous.sum().sum())

## A discard

In [ ]:
# create AnnData using spliced and unspliced count matrix
adata = anndata.AnnData(X = spliced, 
                        layers = dict(spliced = spliced, 
                                    unspliced = unspliced))

adata.obs = example_adata.obs
adata.obsm['X_umap'] = example_adata.obsm['X_umap']

adata.write('anndata/pancreas_usa_trimmed_A_discard.h5ad', compression='gzip')
del(adata)

## A to S:U

In [ ]:
s_ratio = spliced/(spliced+unspliced)
s_ratio = s_ratio.fillna(0.5)
new_spliced = spliced + s_ratio * ambiguous
new_unspliced = unspliced + (1-s_ratio)* ambiguous

adata = anndata.AnnData(X = new_spliced, 
                        layers = dict(spliced = new_spliced, 
                                    unspliced = new_unspliced))

adata.obs = example_adata.obs
adata.write('anndata/pancreas_usa_trimmed_A_S2U.h5ad', compression='gzip')
del(s_ratio, new_spliced, new_unspliced, adata)


## A to S+A:U 

In [ ]:
s_ratio = (spliced+ambiguous)/(spliced+ambiguous+unspliced)
s_ratio = s_ratio.fillna(0.5)
new_spliced = spliced + s_ratio * ambiguous
new_unspliced = unspliced + (1-s_ratio)* ambiguous

adata = anndata.AnnData(X = new_spliced, 
                        layers = dict(spliced = new_spliced, 
                                    unspliced = new_unspliced))
adata.obs = example_adata.obs
adata.obsm['X_umap'] = example_adata.obsm['X_umap']
adata.write('anndata/pancreas_usa_trimmed_A_S+A2U.h5ad', compression='gzip')
del(s_ratio, new_spliced, new_unspliced, adata)


## A to S:U+A 

In [ ]:
s_ratio = (spliced)/(spliced+ambiguous+unspliced)
s_ratio = s_ratio.fillna(0.5)
new_spliced = spliced + s_ratio * ambiguous
new_unspliced = unspliced + (1-s_ratio)* ambiguous

adata = anndata.AnnData(X = new_spliced, 
                        layers = dict(spliced = new_spliced, 
                                    unspliced = new_unspliced))
adata.obs = example_adata.obs
adata.obsm['X_umap'] = example_adata.obsm['X_umap']
adata.write('anndata/pancreas_usa_trimmed_A_S2U+A.h5ad', compression='gzip')
del(s_ratio, new_spliced, new_unspliced, adata)


## A to S

In [ ]:
new_spliced = spliced + ambiguous

adata = anndata.AnnData(X = new_spliced, 
                        layers = dict(spliced = new_spliced, 
                                    unspliced = unspliced))
adata.obs = example_adata.obs
adata.obsm['X_umap'] = example_adata.obsm['X_umap']
adata.write('anndata/pancreas_usa_trimmed_A_S.h5ad', compression='gzip')
del(new_spliced, adata)


## A to U

In [ ]:
new_unspliced = unspliced + ambiguous

adata = anndata.AnnData(X = spliced, 
                        layers = dict(spliced = spliced, 
                                    unspliced = new_unspliced))
adata.obs = example_adata.obs
adata.obsm['X_umap'] = example_adata.obsm['X_umap']
adata.write('anndata/pancreas_usa_trimmed_A_U.h5ad', compression='gzip')
del(new_unspliced, adata)


## A uniform

In [ ]:
s_ratio = 0.5
new_spliced = spliced + s_ratio * ambiguous
new_unspliced = unspliced + (1-s_ratio)* ambiguous

adata = anndata.AnnData(X = new_spliced, 
                        layers = dict(spliced = new_spliced, 
                                    unspliced = new_unspliced))
subset_adata.obs = example_adata.obs
subset_adata.obsm['X_umap'] = example_adata.obsm['X_umap']
subset_adata.write('anndata/pancreas_usa_trimmed_A_unif.h5ad', compression='gzip')
del(s_ratio, new_spliced, new_unspliced, adata, subset_adata)


# Running scVelo

## discard A


In [ ]:
adata = scv.read("anndata/pancreas_usa_trimmed_A_discard.h5ad")
# get embeddings
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.tsne(adata)
# sc.tl.umap(adata, n_components = 2)
adata.obsm['X_umap'] = example_adata.obsm['X_umap']

# housekeeping
matplotlib.use('AGG')
scv.settings.set_figure_params('scvelo')

# get the proportion of spliced and unspliced count
scv.utils.show_proportions(adata)

# filter cells and genes, then normalize expression values
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000,enforce=True)

# scVelo pipeline
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
scv.tl.recover_dynamics(adata, n_jobs = 11)
scv.tl.velocity(adata, mode = 'dynamical')
scv.tl.velocity_graph(adata)
scv.pl.velocity_embedding_stream(adata, basis='umap', save="umap_pancreas_usa_A_discard.png")
scv.pl.velocity_embedding_stream(adata, basis='tsne', save="tsne_pancreas_usa_A_discard.png")
scv.tl.latent_time(adata)
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot', size=80, save = "latent_time_pancreas_usa_A_discard.png")


## A to S

In [ ]:
adata = scv.read("anndata/pancreas_usa_trimmed_A_S.h5ad")
# get embeddings
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.tsne(adata)
# sc.tl.umap(adata, n_components = 2)
adata.obsm['X_umap'] = example_adata.obsm['X_umap']

# housekeeping
matplotlib.use('AGG')
scv.settings.set_figure_params('scvelo')

# get the proportion of spliced and unspliced count
scv.utils.show_proportions(adata)

# filter cells and genes, then normalize expression values
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000,enforce=True)

# scVelo pipeline
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
scv.tl.recover_dynamics(adata, n_jobs = 11)
scv.tl.velocity(adata, mode = 'dynamical')
scv.tl.velocity_graph(adata)
scv.pl.velocity_embedding_stream(adata, basis='umap', save="umap_pancreas_usa_A_S.png")
scv.pl.velocity_embedding_stream(adata, basis='tsne', save="tsne_pancreas_usa_A_S.png")
# scv.pl.velocity_embedding(adata, basis='umap', save="test.png")
scv.tl.latent_time(adata)
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot', size=80, save = "latent_time_pancreas_usa_A_S.png")


## A to U

In [ ]:
adata = scv.read("anndata/pancreas_usa_trimmed_A_U.h5ad")
# get embeddings
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.tsne(adata)
# sc.tl.umap(adata, n_components = 2)
adata.obsm['X_umap'] = example_adata.obsm['X_umap']

# housekeeping
matplotlib.use('AGG')
scv.settings.set_figure_params('scvelo')

# get the proportion of spliced and unspliced count
scv.utils.show_proportions(adata)

# filter cells and genes, then normalize expression values
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000,enforce=True)

# scVelo pipeline
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
scv.tl.recover_dynamics(adata, n_jobs = 11)
scv.tl.velocity(adata, mode = 'dynamical')
scv.tl.velocity_graph(adata)
scv.pl.velocity_embedding_stream(adata, basis='umap', save="umap_pancreas_usa_A_U.png")
scv.pl.velocity_embedding_stream(adata, basis='tsne', save="tsne_pancreas_usa_A_U.png")
# scv.pl.velocity_embedding(adata, basis='umap', save="test.png")
scv.tl.latent_time(adata)
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot', size=80, save = "latent_time_pancreas_usa_A_U.png")


## A to S:U


In [ ]:
adata = scv.read("anndata/pancreas_usa_trimmed_A_S2U.h5ad")
# get embeddings
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.tsne(adata)
# sc.tl.umap(adata, n_components = 2)
adata.obsm['X_umap'] = example_adata.obsm['X_umap']

# housekeeping
matplotlib.use('AGG')
scv.settings.set_figure_params('scvelo')

# get the proportion of spliced and unspliced count
scv.utils.show_proportions(adata)

# filter cells and genes, then normalize expression values
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000,enforce=True)

# scVelo pipeline
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
scv.tl.recover_dynamics(adata, n_jobs = 11)
scv.tl.velocity(adata, mode = 'dynamical')
scv.tl.velocity_graph(adata)
scv.pl.velocity_embedding_stream(adata, basis='umap', save="umap_pancreas_usa_A_S2U.png")
scv.pl.velocity_embedding_stream(adata, basis='tsne', save="tsne_pancreas_usa_A_S2U.png")
scv.tl.latent_time(adata)
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot', size=80, save = "latent_time_pancreas_usa_A_S2U.png")


## A to S+A:U


In [ ]:
adata = scv.read("anndata/pancreas_usa_trimmed_A_S+A2U.h5ad")
# get embeddings
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.tsne(adata)
# sc.tl.umap(adata, n_components = 2)
adata.obsm['X_umap'] = example_adata.obsm['X_umap']

# housekeeping
matplotlib.use('AGG')
scv.settings.set_figure_params('scvelo')

# get the proportion of spliced and unspliced count
scv.utils.show_proportions(adata)

# filter cells and genes, then normalize expression values
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000,enforce=True)

# scVelo pipeline
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
scv.tl.recover_dynamics(adata, n_jobs = 11)
scv.tl.velocity(adata, mode = 'dynamical')
scv.tl.velocity_graph(adata)
scv.pl.velocity_embedding_stream(adata, basis='umap', save="umap_pancreas_usa_A_S+A2U.png")
scv.pl.velocity_embedding_stream(adata, basis='tsne', save="tsne_pancreas_usa_A_S+A2U.png")
scv.tl.latent_time(adata)
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot', size=80, save = "latent_time_pancreas_usa_A_S+A2U.png")


## A to S:U+A


In [ ]:
adata = scv.read("anndata/pancreas_usa_trimmed_A_S2U+A.h5ad")
# get embeddings
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.tsne(adata)
# sc.tl.umap(adata, n_components = 2)
adata.obsm['X_umap'] = example_adata.obsm['X_umap']

# housekeeping
matplotlib.use('AGG')
scv.settings.set_figure_params('scvelo')

# get the proportion of spliced and unspliced count
scv.utils.show_proportions(adata)

# filter cells and genes, then normalize expression values
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000,enforce=True)

# scVelo pipeline
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
scv.tl.recover_dynamics(adata, n_jobs = 11)
scv.tl.velocity(adata, mode = 'dynamical')
scv.tl.velocity_graph(adata)
scv.pl.velocity_embedding_stream(adata, basis='umap', save="umap_pancreas_usa_A_S2U+A.png")
scv.pl.velocity_embedding_stream(adata, basis='tsne', save="tsne_pancreas_usa_A_S2U+A.png")
scv.tl.latent_time(adata)
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot', size=80, save = "latent_time_pancreas_usa_A_S2U+A.png")


## A to uniform


In [ ]:
adata = scv.read("anndata/pancreas_usa_trimmed_A_unif.h5ad")
# get embeddings
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.tsne(adata)
# sc.tl.umap(adata, n_components = 2)
adata.obsm['X_umap'] = example_adata.obsm['X_umap']

# housekeeping
matplotlib.use('AGG')
scv.settings.set_figure_params('scvelo')

# get the proportion of spliced and unspliced count
scv.utils.show_proportions(adata)

# filter cells and genes, then normalize expression values
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000,enforce=True)

# scVelo pipeline
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
scv.tl.recover_dynamics(adata, n_jobs = 11)
scv.tl.velocity(adata, mode = 'dynamical')
scv.tl.velocity_graph(adata)
scv.pl.velocity_embedding_stream(adata, basis='umap', save="umap_pancreas_usa_A_unif.png")
scv.pl.velocity_embedding_stream(adata, basis='tsne', save="tsne_pancreas_usa_A_unif.png")
scv.tl.latent_time(adata)
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot', size=80, save = "latent_time_pancreas_usa_A_unif.png")


## A to S

In [ ]:
adata = scv.read("anndata/pancreas_usa_trimmed_A_S.h5ad")
del adata.obs
# get embeddings
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.tsne(adata)
sc.tl.umap(adata, n_components = 2)
# adata.obsm['X_umap'] = example_adata.obsm['X_umap']

# housekeeping
matplotlib.use('AGG')
scv.settings.set_figure_params('scvelo')

# get the proportion of spliced and unspliced count
scv.utils.show_proportions(adata)

# filter cells and genes, then normalize expression values
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000,enforce=True)

# scVelo pipeline
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
scv.tl.recover_dynamics(adata, n_jobs = 11)
scv.tl.velocity(adata, mode = 'dynamical')
scv.tl.velocity_graph(adata)
scv.pl.velocity_embedding_stream(adata, basis='umap', save="umap_pancreas_usa_A_S_self_embedding.png")
scv.pl.velocity_embedding_stream(adata, basis='tsne', save="tsne_pancreas_usa_A_S_self_embedding.png")
# scv.pl.velocity_embedding(adata, basis='umap', save="test.png")
scv.tl.latent_time(adata)
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot', size=80, save = "latent_time_pancreas_usa_A_S_self_embedding.png")


# example dataset

In [ ]:
example_adata = scv.datasets.pancreas()
# housekeeping
matplotlib.use('AGG')
scv.settings.set_figure_params('scvelo')

# get the proportion of spliced and unspliced count
scv.utils.show_proportions(example_adata)

# filter cells and genes, then normalize expression values
scv.pp.filter_and_normalize(example_adata, min_shared_counts=20, n_top_genes=2000,enforce=True)

# scVelo pipeline
scv.pp.moments(example_adata, n_pcs=30, n_neighbors=30)
scv.tl.recover_dynamics(example_adata, n_jobs = 5)
scv.tl.velocity(example_adata, mode = 'dynamical')
scv.tl.velocity_graph(example_adata)
scv.pl.velocity_embedding_stream(example_adata, basis='umap', save="umap_pancreas_scveloExample.png")
scv.tl.latent_time(example_adata)
scv.pl.scatter(example_adata, color='latent_time', color_map='gnuplot', size=80, save = "latent_time_pancreas_scveloExample.png")
